# Natural Language Processing. Assignment 1. Tokenization.

In this assignment, you need to implement, train, and analyze a Byte-Pair Encoding (BPE) tokenizer.

The assignment consist of 3 tasks. When you finish all the tasks, create a GitHub repository for this assignment (you can use this repository later for the other assignments) and submit this notebook in the repository. Leave `requirements.txt` file if your code requires additional installations. Submit the link to the repository in Moodle.

## Task 1: Data Preparation and Vocabulary Size Selection (3 points)

First, load the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus). After loading the corpus, you need to select the appropriate vocabulary size for the BPE tokenizer. The appropriate vocabulary size is the minimal vocabulary size that covers at least 90% of the words in the corpus. The coverage is calculated according to the following formula:



$$ \text{coverage}(k) = \frac{\sum_{r=1}^{k} f(r)}{\sum_{r=1}^{N} f(r)} $$



where $f(r)$ is the frequency of the top-$r$ word, $k$ is the number of top-$k$ tokens included in vocab, $N$ is the total unique words in corpus.

So, for this task you need to do the following:

1. Load the Brown corpus (0.5 points)
2. Plot cumulative coverage vs. vocabulary size for the loaded corpus (1 point)
3. Select the appropriate vocabulary size (0.5 point)
4. Answer the questions:
    1. Why the coverage slows down the increase as the vocabulary size increases? (0.5 point)
    2. Which empirical law explains the slowing down increase of the coverage? (0.5 point)

In [ ]:
import nltk
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

# Download Brown corpus if not already downloaded
try:
    nltk.data.find('corpora/brown')
except LookupError:
    nltk.download('brown', quiet=True)

from nltk.corpus import brown

# 1. Load the Brown corpus
print("Loading Brown corpus...")
words = brown.words()
print(f"Total words in corpus: {len(words):,}")
print(f"Unique words in corpus: {len(set(words)):,}")

# 2. Calculate and plot cumulative coverage vs vocabulary size
def calculate_coverage(words, vocab_size):
    """
    Calculate cumulative coverage for a given vocabulary size.

    Coverage formula: coverage(k) = sum(frequencies of top-k words) / sum(all frequencies)

    This tells us what percentage of the corpus is covered by the top-k most frequent words.
    For example, if coverage(100) = 0.5, it means the top 100 words account for 50% of all
    word occurrences in the corpus.
    """
    # Step 1: Count how many times each word appears in the corpus
    # Counter creates a dictionary: {word: frequency}
    word_freq = Counter(words)

    # Step 2: Calculate total frequency - sum of all word occurrences
    # This is the denominator in our coverage formula
    total_freq = sum(word_freq.values())

    # Step 3: Get the top-k most frequent words
    # most_common(vocab_size) returns a list of tuples: [(word, freq), ...]
    top_k_words = word_freq.most_common(vocab_size)

    # Step 4: Sum up the frequencies of these top-k words
    # This is the numerator in our coverage formula
    top_k_freq = sum(freq for _, freq in top_k_words)

    # Step 5: Calculate coverage as a ratio
    coverage = top_k_freq / total_freq if total_freq > 0 else 0.0
    return coverage

# Calculate coverage for different vocabulary sizes
word_freq = Counter(words)
total_unique_words = len(word_freq)

vocab_sizes = []
coverages = []

# Adaptive step size: use smaller steps when coverage changes rapidly (small vocab sizes),
# and larger steps when coverage plateaus (large vocab sizes)
# This gives us more data points where the curve is interesting (steep) and fewer
# where it's flat, making the plot more informative
current_size = 0
while current_size < min(10000, total_unique_words):
    # Determine step size based on current vocabulary size
    # Small vocab sizes: coverage changes quickly, so we sample more frequently (step=1)
    if current_size < 100:
        step = 1
    # Medium-small: still changing, sample every 5 words
    elif current_size < 500:
        step = 5
    # Medium: changes slower, sample every 10 words
    elif current_size < 1000:
        step = 10
    # Medium-large: changes very slowly, sample every 50 words
    elif current_size < 5000:
        step = 50
    # Large: coverage barely changes, sample every 100 words
    else:
        step = 100

    # Move to next vocabulary size
    current_size += step
    # Don't exceed the total number of unique words
    if current_size > total_unique_words:
        current_size = total_unique_words

    # Calculate coverage for this vocabulary size
    coverage = calculate_coverage(words, current_size)
    # Store the data point
    vocab_sizes.append(current_size)
    coverages.append(coverage)

    # Stop if we've covered all unique words
    if current_size >= total_unique_words:
        break

# Plot cumulative coverage vs vocabulary size
plt.figure(figsize=(10, 6))
plt.plot(vocab_sizes, coverages, 'b-', linewidth=2)
plt.axhline(y=0.9, color='r', linestyle='--', label='90% Coverage')
plt.xlabel('Vocabulary Size', fontsize=12)
plt.ylabel('Cumulative Coverage', fontsize=12)
plt.title('Cumulative Coverage vs Vocabulary Size (Brown Corpus)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# 3. Select the appropriate vocabulary size (minimal that covers at least 90%)
def find_appropriate_vocab_size(words, target_coverage=0.9):
    """
    Find minimal vocabulary size that covers at least target_coverage.

    Uses binary search to efficiently find the answer. Instead of checking every possible
    vocabulary size (which could be thousands), we use binary search to find the answer
    in O(log n) time.
    """
    # Step 1: Count word frequencies and get total unique words
    word_freq = Counter(words)
    total_unique_words = len(word_freq)

    # Step 2: Binary search setup
    # We search in the range [1, total_unique_words]
    # left = smallest possible vocab size (1 word)
    # right = largest possible vocab size (all unique words)
    left, right = 1, total_unique_words
    result = total_unique_words  # Default: use all words if we can't find a smaller solution

    # Step 3: Binary search loop
    # The idea: coverage increases as vocab_size increases (more words = more coverage)
    # So we can use binary search to find the smallest vocab_size with coverage >= 0.9
    while left <= right:
        # Check the middle point
        mid = (left + right) // 2
        # Calculate coverage for this vocabulary size
        coverage = calculate_coverage(words, mid)

        # If we've reached our target coverage (e.g., >= 90%)
        if coverage >= target_coverage:
            # This vocab_size works! But maybe a smaller one also works?
            # So we record this as a candidate and search left (smaller vocab sizes)
            result = mid
            right = mid - 1  # Try smaller vocab sizes
        else:
            # Coverage is too low, we need more words
            # So we search right (larger vocab sizes)
            left = mid + 1

    # Return the minimal vocabulary size that achieves target coverage
    return result

vocab_size = find_appropriate_vocab_size(words, target_coverage=0.9)
actual_coverage = calculate_coverage(words, vocab_size)

print(f"\nAppropriate vocabulary size: {vocab_size}")
print(f"Actual coverage achieved: {actual_coverage:.4f} ({actual_coverage*100:.2f}%)")

# 4. Answer the questions
print("\n" + "="*60)
print("ANSWERS TO QUESTIONS:")
print("="*60)
print("\n1. Why does the coverage slow down as vocabulary size increases?")
print("   Answer: The coverage slows down because of Zipf's law - word")
print("   frequencies follow a power-law distribution. The most frequent")
print("   words (top-ranked) account for a large portion of the corpus,")
print("   while less frequent words (lower-ranked) appear much less often.")
print("   As we add more words to the vocabulary, we're adding words with")
print("   progressively lower frequencies, so each additional word contributes")
print("   less to the total coverage.")

print("\n2. Which empirical law explains the slowing down increase of coverage?")
print("   Answer: Zipf's law (or Zipf's distribution) explains this phenomenon.")
print("   Zipf's law states that the frequency of a word is inversely")
print("   proportional to its rank in the frequency table. This means that")
print("   a small number of high-frequency words account for most of the")
print("   corpus, while many low-frequency words account for very little.")
print("   This power-law distribution causes the coverage curve to flatten")
print("   as vocabulary size increases.")

Loading Brown corpus...
Total words in corpus: 1,161,192
Unique words in corpus: 56,057


## Task 2: Implement Byte-Pair Encoding (BPE) Tokenizer (4 points)

Implement the [BPE tokenizer](https://arxiv.org/pdf/1508.07909) as the `BPETokenizer` class.

The class should contain correctly implemented:

* `train` method (1.5 points).
* `tokenize` method (1.5 points).

The code should have docstrings and comments (1 point).

In [ ]:
"""
Byte-Pair Encoding (BPE) Tokenizer Implementation

BPE is a data compression algorithm adapted for NLP subword tokenization.
"""

import re
from collections import Counter, defaultdict
from typing import List, Tuple, Dict


class BPETokenizer:
    """
    Byte-Pair Encoding (BPE) Tokenizer implementation.

    BPE iteratively replaces the most frequent pair of consecutive subword units
    with a single, unused token. This allows handling of rare and out-of-vocabulary words.
    """

    def __init__(self):
        """Initialize the BPE tokenizer."""
        # Vocabulary: maps token to its index
        self.vocab = {}

        # Reverse vocabulary: maps index to token
        self.vocab_reverse = {}

        # BPE merges: list of (token1, token2) pairs that were merged
        self.merges = []

        # Special tokens
        self.unk_token = "<UNK>"
        self.end_of_word_token = "</w>"  # Marks end of word

    def _pre_tokenize(self, text: str) -> List[str]:
        """
        Pre-tokenize text into words and add end-of-word markers.

        This is the first step in BPE tokenization. We need to:
        1. Split text into words (pre-tokenization)
        2. Add a special marker to indicate word boundaries (</w>)

        Why add </w>? BPE works on character/subword level, so we need to know where
        words end. Otherwise, "cat" and "cats" might merge incorrectly with other words.

        Args:
            text: Input text string

        Returns:
            List of pre-tokenized words with end-of-word markers
        """
        # Step 1: Split text into words
        # \S+ matches one or more non-whitespace characters (words)
        # .lower() converts to lowercase for consistency
        words = re.findall(r'\S+', text.lower())

        # Step 2: Add end-of-word marker to each word
        # This marker helps BPE know where words end during merging
        words = [word + self.end_of_word_token for word in words]

        return words

    def _get_word_freqs(self, texts: List[str]) -> Dict[str, int]:
        """
        Get word frequencies from a list of texts.

        Args:
            texts: List of text strings

        Returns:
            Dictionary mapping words (with </w>) to their frequencies
        """
        word_freqs = Counter()

        for text in texts:
            words = self._pre_tokenize(text)
            word_freqs.update(words)

        return dict(word_freqs)

    def _get_stats(self, word_freqs: Dict[str, int]) -> Dict[Tuple[str, str], int]:
        """
        Get statistics of pairs of consecutive symbols.

        This is the core of BPE: we need to find which pairs of consecutive characters/subwords
        appear most frequently. These frequent pairs will be merged into a single token.

        Example: If "th" appears 1000 times and "he" appears 800 times, we'll merge "th" first.

        Args:
            word_freqs: Dictionary mapping words to their frequencies

        Returns:
            Dictionary mapping pairs (char1, char2) to their total frequency
        """
        pairs = defaultdict(int)

        for word, freq in word_freqs.items():
            # Step 1: Split word into individual characters/symbols
            symbols = list(word)

            # Step 2: Count all pairs of consecutive symbols
            # We iterate through adjacent pairs and increment count by word frequency
            for i in range(len(symbols) - 1):
                pairs[(symbols[i], symbols[i + 1])] += freq

        return pairs

    def _merge_vocab(self, pair: Tuple[str, str], word_freqs: Dict[str, int]) -> Dict[str, int]:
        """
        Merge the most frequent pair in the vocabulary.

        Args:
            pair: Tuple of (token1, token2) to merge
            word_freqs: Current word frequencies dictionary

        Returns:
            Updated word frequencies dictionary with merged pairs
        """
        new_word_freqs = {}
        merged_token = ''.join(pair)

        for word in word_freqs:
            # Replace consecutive occurrences of the pair with merged token
            # We need to replace all occurrences, not just one
            new_word = word.replace(''.join(pair), merged_token)
            new_word_freqs[new_word] = word_freqs[word]

        return new_word_freqs

    def train(self, texts: List[str], vocab_size: int):
        """
        Train the BPE tokenizer on a list of texts.

        The training process:
        1. Start with character-level vocabulary (each character is a token)
        2. Iteratively find the most frequent pair of consecutive tokens
        3. Merge that pair into a single token
        4. Repeat until we reach the desired vocabulary size

        Args:
            texts: List of training text strings
            vocab_size: Desired vocabulary size (including special tokens)
        """
        # Step 1: Pre-tokenize texts and count word frequencies
        word_freqs = self._get_word_freqs(texts)

        # Step 2: Initialize vocabulary with all unique characters
        # BPE starts at the character level - each character is initially a separate token
        vocab = set()
        for word in word_freqs.keys():
            for char in word:
                vocab.add(char)  # Add each unique character

        # Step 3: Convert to sorted list for consistent ordering
        vocab = sorted(list(vocab))

        # Step 4: Add special tokens (like <UNK> for unknown words)
        if self.unk_token not in vocab:
            vocab.insert(0, self.unk_token)

        # Step 5: Create vocabulary dictionaries
        # vocab: maps token -> index
        # vocab_reverse: maps index -> token
        self.vocab = {token: idx for idx, token in enumerate(vocab)}
        self.vocab_reverse = {idx: token for token, idx in self.vocab.items()}

        # Step 6: Calculate how many merges we need to perform
        # If we want vocab_size=1000 and we have 100 characters, we need 900 merges
        num_merges = vocab_size - len(self.vocab)

        # Step 7: Perform BPE merges iteratively
        self.merges = []  # Store the order of merges (important for tokenization later)

        for i in range(num_merges):
            # Step 7a: Get statistics of all pairs of consecutive tokens
            pairs = self._get_stats(word_freqs)

            if not pairs:
                break

            # Step 7b: Find the most frequent pair
            best_pair = max(pairs, key=pairs.get)

            # Step 7c: Merge this pair in all words
            word_freqs = self._merge_vocab(best_pair, word_freqs)

            # Step 7d: Add the merged token to our vocabulary
            merged_token = ''.join(best_pair)
            if merged_token not in self.vocab:
                self.vocab[merged_token] = len(self.vocab)
                self.vocab_reverse[len(self.vocab) - 1] = merged_token

            # Step 7e: Record this merge (we need this order for tokenization)
            self.merges.append(best_pair)

    def _apply_bpe(self, word: str) -> List[str]:
        """
        Apply BPE merges to a single word.

        This function takes a word and applies all the learned BPE merges in order.
        It's like "replaying" the training process but for a single word.

        Args:
            word: Input word (should include </w> marker)

        Returns:
            List of BPE tokens
        """
        # Step 1: Ensure word has end-of-word marker
        word = word + self.end_of_word_token if not word.endswith(self.end_of_word_token) else word

        # Step 2: Start with individual characters
        # This is our initial state - each character is a separate token
        tokens = list(word)

        # Step 3: Apply all learned merges in the same order they were learned
        # This is crucial! The order matters because later merges depend on earlier ones
        for pair in self.merges:
            new_tokens = []  # Will store tokens after applying this merge
            i = 0  # Index to traverse current tokens

            # Go through tokens and look for the pair to merge
            while i < len(tokens):
                # Check if current token and next token form the pair we want to merge
                if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i + 1] == pair[1]:
                    # Found the pair! Merge them into a single token
                    new_tokens.append(''.join(pair))
                    i += 2  # Skip both tokens since we merged them
                else:
                    # No merge here, keep the token as is
                    new_tokens.append(tokens[i])
                    i += 1  # Move to next token

            # Update tokens for next iteration
            tokens = new_tokens

        return tokens

    def tokenize(self, text: str) -> List[str]:
        """
        Tokenize a text string using the trained BPE tokenizer.

        Args:
            text: Input text string

        Returns:
            List of tokenized subwords
        """
        # Pre-tokenize into words
        words = self._pre_tokenize(text)

        # Apply BPE to each word
        tokens = []
        for word in words:
            word_tokens = self._apply_bpe(word)
            tokens.extend(word_tokens)

        return tokens

    def encode(self, text: str) -> List[int]:
        """
        Encode text to token IDs.

        Args:
            text: Input text string

        Returns:
            List of token IDs
        """
        tokens = self.tokenize(text)
        token_ids = []

        for token in tokens:
            if token in self.vocab:
                token_ids.append(self.vocab[token])
            else:
                # Handle unknown tokens
                token_ids.append(self.vocab.get(self.unk_token, 0))

        return token_ids

    def decode(self, token_ids: List[int]) -> str:
        """
        Decode token IDs back to text.

        Args:
            token_ids: List of token IDs

        Returns:
            Decoded text string
        """
        tokens = [self.vocab_reverse.get(idx, self.unk_token) for idx in token_ids]

        # Remove end-of-word markers and join
        text = ''.join(tokens)
        text = text.replace(self.end_of_word_token, ' ')

        return text.strip()


# Test the implementation
print("BPE Tokenizer Implementation")
print("=" * 50)

# Sample training texts for demonstration
sample_texts = [
    "The quick brown fox jumps over the lazy dog.",
    "Natural language processing is fascinating.",
    "Tokenization is an important preprocessing step."
]

# Initialize tokenizer
tokenizer = BPETokenizer()

# Train with small vocabulary size for demonstration
print("\nTraining tokenizer...")
tokenizer.train(sample_texts, vocab_size=50)

print(f"Vocabulary size: {len(tokenizer.vocab)}")
print(f"Number of merges: {len(tokenizer.merges)}")

# Test tokenization
test_text = "The quick brown fox"
tokens = tokenizer.tokenize(test_text)
token_ids = tokenizer.encode(test_text)

print(f"\nTest text: '{test_text}'")
print(f"Tokens: {tokens}")
print(f"Token IDs: {token_ids}")

## Task 3: Tokenizer Training and Analysis (3 points)

1. Train the `BPETokenizer` on the Brown corpus with the appropriate vocabulary size selected in Task 1 (1 points)
2. Use the Brown corpus (1000 samples) to calculate the mean and standard deviation of
    * tokenizer's fertility (1 points)
    * length of the tokenized sentence (1 points)

In [ ]:
import numpy as np

# 1. Train the BPETokenizer on the Brown corpus with the appropriate vocabulary size
print("=" * 60)
print("Task 3: Tokenizer Training and Analysis")
print("=" * 60)

# Load Brown corpus for training
print("\n1. Loading Brown corpus for training...")
all_sentences = brown.sents()
training_texts = [' '.join(sent) for sent in all_sentences]

print(f"   Total training sentences: {len(training_texts):,}")

# Use vocabulary size from Task 1
print(f"\n2. Training BPE tokenizer with vocabulary size: {vocab_size}")
tokenizer = BPETokenizer()
tokenizer.train(training_texts, vocab_size=vocab_size)

print(f"   Actual vocabulary size: {len(tokenizer.vocab)}")
print(f"   Number of BPE merges: {len(tokenizer.merges)}")

# 2. Use Brown corpus (1000 samples) to calculate statistics
print(f"\n3. Getting 1000 sample sentences for analysis...")
sample_sentences = [' '.join(sent) for sent in brown.sents()[:1000]]

# Calculate fertility (tokens per word)
def calculate_fertility(tokenizer, sentences):
    """
    Calculate fertility: ratio of tokens to words.

    Fertility measures how many tokens the tokenizer produces per word.
    - Fertility = 1.0 means each word becomes exactly 1 token
    - Fertility > 1.0 means words are split into multiple tokens (subword tokenization)
    - Higher fertility = more tokens = potentially more information but also more computation
    """
    fertilities = []
    for sentence in sentences:
        # Count original words in the sentence
        num_words = len(sentence.split())

        # Tokenize the sentence using BPE (splits words into subword tokens)
        tokens = tokenizer.tokenize(sentence)
        num_tokens = len(tokens)

        # Calculate fertility ratio
        if num_words > 0:
            fertility = num_tokens / num_words
        else:
            fertility = 0.0  # Handle edge case: empty sentence
        fertilities.append(fertility)
    return np.array(fertilities)

# Calculate tokenized sentence length
def calculate_tokenized_length(tokenizer, sentences):
    """Calculate length of tokenized sentences."""
    lengths = []
    for sentence in sentences:
        tokens = tokenizer.tokenize(sentence)
        lengths.append(len(tokens))
    return np.array(lengths)

print("\n4. Calculating fertility statistics...")
fertilities = calculate_fertility(tokenizer, sample_sentences)
fertility_mean = np.mean(fertilities)
fertility_std = np.std(fertilities)

print(f"   Mean fertility: {fertility_mean:.4f}")
print(f"   Std fertility: {fertility_std:.4f}")

print("\n5. Calculating tokenized sentence length statistics...")
lengths = calculate_tokenized_length(tokenizer, sample_sentences)
length_mean = np.mean(lengths)
length_std = np.std(lengths)

print(f"   Mean tokenized sentence length: {length_mean:.2f}")
print(f"   Std tokenized sentence length: {length_std:.2f}")

# Print summary
print("\n" + "=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)
print(f"Vocabulary Size: {len(tokenizer.vocab)}")
print(f"\nFertility (tokens/word):")
print(f"  Mean: {fertility_mean:.4f}")
print(f"  Std:  {fertility_std:.4f}")
print(f"\nTokenized Sentence Length:")
print(f"  Mean: {length_mean:.2f}")
print(f"  Std:  {length_std:.2f}")

## Grading Procedure Details

During the grading of the completed assignments, a random set of students will be sampled for the **offline assignment defence**. The defence will be arranged shortly after the assignment submission deadline. The particular date and time will be announced later. 

The aim of the assignment defence is to ensure the students understand well their own solutions and know how thier solution works. To check this, the students will be asked various questions about the provided solution. In addition, the students will be asked to run their solution to ensure the solution works without errors.

Examples of questions:

1. How the cumulative coverage is calculated? Why is it called cumulative?
2. What is the rank of a word?
3. How does the BPE tokenizer work? Note: for this question, the students will not be able to see the their own implementation.
4. Why do you consider such vocabulary size appropriate?
5. What is the formula for the fertility of the tokenizer?
6. How do you perform pre-tokenization in your implementation?
7. How do you handle stopwords in the training corpus? Why?
8. etc.

As a result of the assignment defence, the grade for the assignment may be adjusted.